## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [7]:

classifier=Sequential()

#卷積組合  , filters =32 , kernel_size= (3,3), strides=(1, 1), padding='same'
##Kernel size 3*3，用32張，輸入大小32*32*3
classifier.add(Convolution2D(filters =32 , kernel_size= (3,3), strides=(1, 1), padding='same',input_shape=(32,32,3)))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
#classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3))) #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters =32 , kernel_size= (3,3), strides=(1, 1), padding='same')) #
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

print(classifier.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 16, 16, 32)       

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [8]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 16s 324us/step - loss: 1.3322 - accuracy: 0.5256
Epoch 2/100
50000/50000 [==============================] - 10s 206us/step - loss: 0.9723 - accuracy: 0.6579
Epoch 3/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.8451 - accuracy: 0.7033
Epoch 4/100
50000/50000 [==============================] - 10s 198us/step - loss: 0.7676 - accuracy: 0.7332
Epoch 5/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.7042 - accuracy: 0.7529
Epoch 6/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.6444 - accuracy: 0.7750
Epoch 7/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.5977 - accuracy: 0.7896
Epoch 8/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.5487 - accuracy: 0.8071
Epoch 9/100
50000/50000 [==============================] - 10s 199us/step - loss: 0.5075 - accuracy: 0.8234
Epoch 10/100
50000/50000 [==

In [9]:
score_train = classifier.evaluate(x_train, y_train, verbose=1)
print('train loss:', score_train[0])
print('train accuracy:', score_train[1])

50000/50000 [==============================] - 8s 170us/step
train loss: 0.012845843896151055
train accuracy: 0.9961599707603455


In [10]:
scores_test = classifier.evaluate(x_test, y_test, verbose=1)
print('test loss:', scores_test[0])
print('test accuracy:', scores_test[1])

10000/10000 [==============================] - 2s 169us/step
test loss: 3.908729706954956
test accuracy: 0.6535000205039978


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [13]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[9.1663729e-05, 1.4269936e-16, 7.5310445e-06, 9.9922454e-01,
        6.3538289e-04, 1.4607748e-09, 4.8214172e-10, 9.2673097e-10,
        4.0845720e-05, 1.8829894e-15]], dtype=float32)